In [1]:
# # prompt: mount google drive

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !apt-get update && apt-get install -y libsndfile1 ffmpeg
!pip install Cython packaging
!pip install git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[asr]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.4 MB/s eta 0:00:00:00:01
DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[asr] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-74f_odzx/nemo-toolkit_49fc447188f54b8ea1de5f3f13147ece
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-74f_odzx/nemo-toolkit_49fc447188f54b8ea1de5f3f13147ece
  Resolved https://github.com/NVIDIA/NeMo.git to commit 64625d4814c87d545efa533fdc6cf6b62105c23e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of nemo-toolkit[a

In [ ]:
import pandas as pd
import getpass
import torch
import soundfile as sf
from IPython.display import Audio

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/data/afrispeech_dialog_v1_47.csv')

In [ ]:
dataset.shape

(49, 11)

In [ ]:
dataset.head()

,audio_id,path,transcript,age_group,gender,accent,domain,country,audio_duration
0,58efe9ef-6c76-4d17-b1b2-f397552a0c0e,data/medical/95aed576-a1d6-42f3-9651-7103e0717...,00:01:13\r\n[Speaker 1]: Good day. I am Doctor...,26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,213.635986
1,bbc98b6e-f165-4475-b0b3-8e25ab259323,data/medical/9d838ece-8db5-4736-96d8-581071bd5...,"\r\n00:01:25\r\n[Speaker 1]: Hello, how are yo...",26-40,Female,Yoruba,OSCE-Doctor-Patient,NG,216.760998
2,d3550502-5ae7-4314-8a21-036d488cd8d6,data/medical/656c42a7-6faa-4486-979f-6f64769bd...,\r\n00:00:59\r\n[Speaker 1]: Hello. I am docto...,26-40,Female,Yoruba,OSCE-Doctor-Patient,NG,218.482993
3,b6f48ed7-e629-4e50-af63-1c9067c34ef2,data/medical/cd255fe0-ebc2-47a0-a990-2ab1ba9ca...,"00:00:35\r\n[Speaker 1]: Hello, how are you?\r...",26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,220.120998
4,f9bd30b2-684d-470e-b65c-ced156da0bbb,data/medical/304d6402-91d8-4a4f-8a65-143a7c767...,"\r\n\r\n00:04:08\r\n[Speaker 1]: Hi, I am Dr. ...",26-40,Female,Yoruba,OSCE-Doctor-Patient,NG,235.932993


In [ ]:
def merge_consecutive_segments(predicted_segments):
  merged_segments = []
  last_start, last_end, last_speaker = None, None, None

  # predicted_segments is a list of lists of strings, e.g., [['0.000 1.840 speaker_0', ...]]
  # We need to iterate through the inner list of segments
  if predicted_segments and isinstance(predicted_segments[0], list):
    segments_list = predicted_segments[0]
  else:
    # Handle unexpected format or empty list
    return []

  for segment_str in segments_list:
    try:
        start_str, end_str, speaker = segment_str.split()
        start_time = float(start_str)
        end_time = float(end_str)
    except ValueError:
        # Skip segments that don't have the expected format
        print(f"Warning: Skipping segment with unexpected format: {segment_str}")
        continue


    #check if the current segment should be merged with the previous one
    if last_speaker is not None and speaker == last_speaker:
        #extend the end time of the last segment
        last_end = end_time
    else:
        #if it's not the first iteration and there's a previous segment to save
        if last_speaker is not None:
            merged_segments.append((last_start, last_end, last_speaker))
        #update last segment trackers
        last_start, last_end, last_speaker = start_time, end_time, speaker

  #add the last segment after exiting the loop
  if last_speaker is not None:
      merged_segments.append((last_start, last_end, last_speaker))

  return merged_segments

In [ ]:
!pip install --upgrade --force-reinstall torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
  Using cached torch-2.7.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.7.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.meta

In [ ]:
!pip install numpy==2.1.0 --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 86.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nemo-toolkit 2.4.0rc0 requires fsspec==2024.12.0, but you have fsspec 2025.5.0 which is incompatible.
dask-cuda 25.2.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.0 which is incompatible.
cuml-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.0 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.1.0 which is incompatible.
distributed-ucxx-cu12 0.42.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.0 which is incompatible.
fastai 

In [3]:
import numpy as np
import torch
import nemo
import soundfile as sf

print("NumPy version:", np.__version__)
print("Torch version:", torch.__version__)
print("NeMo version:", nemo.__version__)


ModuleNotFoundError: No module named 'nemo'

In [4]:
from nemo.collections.asr.models import SortformerEncLabelModel

# load model from Hugging Face model card directly (You need a Hugging Face token)
diar_model = SortformerEncLabelModel.from_pretrained("nvidia/diar_sortformer_4spk-v1")


# switch to inference mode
diar_model.eval()


ModuleNotFoundError: No module named 'nemo'

In [5]:
audio_input="/content/drive/MyDrive/data/medical/4696599d-37fb-442b-bdae-27e3325771b5_caecc2042dce919b3fa39143f8a9473a_a9KTbuFt.wav"


In [10]:
from IPython.display import Audio
Audio(audio_input)


ValueError: rate must be specified when data is a numpy array or list of audio samples.

In [6]:
import soundfile as sf
audio_signal, sample_rate = sf.read(audio_input)


LibsndfileError: Error opening '/content/drive/MyDrive/data/medical/4696599d-37fb-442b-bdae-27e3325771b5_caecc2042dce919b3fa39143f8a9473a_a9KTbuFt.wav': System error.

In [7]:
import soundfile as sf
import numpy as np

# Read stereo file
audio, sr = sf.read(audio_input)  # audio.shape = (samples, 2)

# Convert to mono by averaging channels
if len(audio.shape) == 2:
    audio = np.mean(audio, axis=1)

# Save as mono WAV
sf.write('mono_audio.wav', audio, sr)


LibsndfileError: Error opening '/content/drive/MyDrive/data/medical/4696599d-37fb-442b-bdae-27e3325771b5_caecc2042dce919b3fa39143f8a9473a_a9KTbuFt.wav': System error.

In [8]:
predicted_segments = diar_model.diarize(audio='mono_audio.wav', batch_size=1)

#pred_segments = merge_consecutive_segments(diarization)

NameError: name 'diar_model' is not defined

In [ ]:
merged_segments = merge_consecutive_segments(predicted_segments)


In [9]:
print(predicted_segments)
unique_speakers = {chunk.split(" ")[2] for chunk in predicted_segments[0]}
print(unique_speakers)

NameError: name 'predicted_segments' is not defined